In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Directorios de entrenamiento y validación
data_entrenamiento = '/content/drive/MyDrive/images_dataset/training'
data_validacion = '/content/drive/MyDrive/images_dataset/validation'

total_imagenes_entrenamiento = 2130  # Reemplaza con el número total de imágenes de entrenamiento
total_imagenes_validacion = 1287  # Reemplaza con el número total de imágenes de validación

# Parámetros
epocas = 20
longitud, altura = 150, 150
batch_size = 32
pasos = total_imagenes_entrenamiento // batch_size
validation_steps = total_imagenes_validacion // batch_size
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 4
lr = 0.0005

# Preprocesamiento de imágenes
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

# Crear modelo CNN
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding="same", activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

# Compilar el modelo
cnn.compile(loss='categorical_crossentropy',
            optimizer=Adam(lr=lr),
            metrics=['accuracy'])

# Entrenar el modelo
cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

cnn.save('/content/modelo/modelo.h5')
cnn.save_weights('/content/modelo/pesos.h5')

Found 2130 images belonging to 4 classes.
Found 1287 images belonging to 4 classes.


Epoch 1/20
66/66 [==============================] - 136s 2s/step - loss: 0.8661 - accuracy: 0.7731 - val_loss: 0.2566 - val_accuracy: 0.8875
Epoch 2/20
66/66 [==============================] - 122s 2s/step - loss: 0.0651 - accuracy: 0.9814 - val_loss: 0.5825 - val_accuracy: 0.8656
Epoch 3/20
66/66 [==============================] - 122s 2s/step - loss: 0.0260 - accuracy: 0.9929 - val_loss: 0.5044 - val_accuracy: 0.8914
Epoch 4/20
66/66 [==============================] - 122s 2s/step - loss: 0.0152 - accuracy: 0.9952 - val_loss: 0.5333 - val_accuracy: 0.8977
Epoch 5/20
66/66 [==============================] - 123s 2s/step - loss: 0.0085 - accuracy: 0.9986 - val_loss: 0.4953 - val_accuracy: 0.9117
Epoch 6/20
66/66 [==============================] - 125s 2s/step - loss: 0.0132 - accuracy: 0.9952 - val_loss: 0.6603 - val_accuracy: 0.8883
Epoch 7/20
66/66 [==============================] - 123s 2s/step - loss: 0.0096 - accuracy: 0.9971 - val_loss: 0.8420 - val_accuracy: 0.8914
Epoch 8/20
66

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

longitud, altura = 150, 150
modelo = '/content/modelo/modelo.h5'
pesos_modelo = '/content/modelo/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Ambulancia")
  elif answer == 1:
    print("pred: Bus")
  elif answer == 2:
    print("pred: Carro de Bomberos")
  elif answer == 3:
    print("pred: Rover")

  return answer

predict('/content/drive/MyDrive/images_dataset/validation/carro_bomberos/image_2871.png')

1/1 [==============================] - 0s 137ms/step
pred: Carro de Bomberos


2

In [ ]:
%%shell
jupyter nbconvert --to html /content/Untitled17.ipynb

[NbConvertApp] Converting notebook /content/Untitled17.ipynb to html
[NbConvertApp] Writing 595851 bytes to /content/Untitled17.html
